In [2]:
import numpy as np
import itertools
import sys
import inspect
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=200)
MAX_INT16 = np.iinfo(np.int16).max;
MAX_INT32 = np.iinfo(np.int32).max;
from IPython.display import display, clear_output

In [3]:
f = open("../input/day-15.input.txt","r")
lines = f.read().splitlines()


initial_levels = np.array(
    list(map(lambda row: 
             list(map(lambda cell: int(cell), row))
        , lines)
    )
)

In [3]:
## testing
test_levels = np.array([
[1,1,6,3,7,5,1,7,4,2],
[1,3,8,1,3,7,3,6,7,2],
[2,1,3,6,5,1,1,3,2,8],
[3,6,9,4,9,3,1,5,6,9],
[7,4,6,3,4,1,7,1,1,1],
[1,3,1,9,1,2,8,1,3,7],
[1,3,5,9,9,1,2,4,2,1],
[3,1,2,5,4,2,1,6,3,9],
[1,2,9,3,1,3,8,5,2,1],
[2,3,1,1,9,4,4,5,8,1],
], np.int32)

In [4]:
test_levels

array([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
       [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
       [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
       [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
       [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
       [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
       [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
       [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
       [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
       [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [5]:
lst = [(0,0), (5,1), (2,200)]
lst.append((2,4))
not len([ind for ind, x in enumerate(lst) if x[1] == 200])
lst.sort(reverse=True)
lst.pop()
lst

[(5, 1), (2, 200), (2, 4)]

In [6]:
def compute_heuristic_map(input_map):
    rows = len(input_map)
    columns = len(input_map[0])
    heuristics = np.zeros((rows, columns), np.int32)
    for r in range(rows):
        for c in range(columns):
            heuristics[r][c] = rows -r + columns - c -2
    return heuristics

In [7]:
test_heuristic_map = compute_heuristic_map(test_levels)
test_heuristic_map

array([[18, 17, 16, 15, 14, 13, 12, 11, 10,  9],
       [17, 16, 15, 14, 13, 12, 11, 10,  9,  8],
       [16, 15, 14, 13, 12, 11, 10,  9,  8,  7],
       [15, 14, 13, 12, 11, 10,  9,  8,  7,  6],
       [14, 13, 12, 11, 10,  9,  8,  7,  6,  5],
       [13, 12, 11, 10,  9,  8,  7,  6,  5,  4],
       [12, 11, 10,  9,  8,  7,  6,  5,  4,  3],
       [11, 10,  9,  8,  7,  6,  5,  4,  3,  2],
       [10,  9,  8,  7,  6,  5,  4,  3,  2,  1],
       [ 9,  8,  7,  6,  5,  4,  3,  2,  1,  0]])

In [8]:
test_heuristic_map_flat = test_heuristic_map.flatten()
test_heuristic_map_flat

array([18, 17, 16, 15, 14, 13, 12, 11, 10,  9, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6, 14, 13, 12, 11, 10,  9,  8,  7,
        6,  5, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  9,  8,  7,  6,  5,  4,
        3,  2,  1,  0])

In [9]:
test_heuristic_map_flat.reshape((len(test_levels),len(test_levels[0])))

array([[18, 17, 16, 15, 14, 13, 12, 11, 10,  9],
       [17, 16, 15, 14, 13, 12, 11, 10,  9,  8],
       [16, 15, 14, 13, 12, 11, 10,  9,  8,  7],
       [15, 14, 13, 12, 11, 10,  9,  8,  7,  6],
       [14, 13, 12, 11, 10,  9,  8,  7,  6,  5],
       [13, 12, 11, 10,  9,  8,  7,  6,  5,  4],
       [12, 11, 10,  9,  8,  7,  6,  5,  4,  3],
       [11, 10,  9,  8,  7,  6,  5,  4,  3,  2],
       [10,  9,  8,  7,  6,  5,  4,  3,  2,  1],
       [ 9,  8,  7,  6,  5,  4,  3,  2,  1,  0]])

In [10]:
def list_contains(l,el):
    return len(l[l == el]) > 0

In [39]:
def a_star(levels_map, goal=None):
    levels_map_flat = levels_map.flatten()
    heuristic_map = compute_heuristic_map(levels_map)
    heuristic_flat_map = heuristic_map.flatten()
    
    row_length = len(levels_map[0])
    col_length = len(levels_map)
    
    if not goal:
        goal = (row_length * col_length) -1
    openlist = [(0,0)] 
    g_values_flat = np.full(col_length*row_length, 0, np.int32)
    g_values_flat[0] = 0
    previous_flat_map = np.full(col_length*row_length, -1, np.int32)
    previous_flat_map[0] = 0
    closedlist  = np.array([], np.int32)

    
    # überprüft alle Nachfolgeknoten und fügt sie der Open List hinzu, wenn entweder
    # - der Nachfolgeknoten zum ersten Mal gefunden wird, oder
    # - ein besserer Weg zu diesem Knoten gefunden wird
    def expand_node(current_node):
        nonlocal openlist
        nonlocal closedlist
        successors = np.array([], np.int32);
        curr_row = int(current_node/len(levels_map))
        curr_col = current_node%len(levels_map)
        if curr_row != 0:
            successors = np.append(successors, (curr_row - 1) * row_length + curr_col)
        if curr_row != len(levels_map)-1:
            successors = np.append(successors, (curr_row + 1)  * row_length + curr_col)     
        if curr_col != 0:
            successors = np.append(successors, curr_row * row_length + curr_col -1)
        if curr_col != len(levels_map[0]) - 1:
            successors = np.append(successors, curr_row * row_length + curr_col + 1)       


        for successor in successors:
            # wenn der Nachfolgeknoten bereits auf der Closed List ist – tue nichts
            if list_contains(closedlist, successor):
                continue
            
            # g-Wert für den neuen Weg berechnen: g-Wert des Vorgängers plus
            # die Kosten der gerade benutzten Kante
            tentative_g = g_values_flat[current_node] + levels_map_flat[current_node]
            
            # wenn der Nachfolgeknoten bereits auf der Open List ist,
            # aber der neue Weg nicht besser ist als der alte – tue nichts
            openlist_succ_index = [i for i, x in enumerate(openlist) if x[1] == successor]
            if len(openlist_succ_index) and tentative_g >= g_values_flat[successor]:
                continue
    
            # Vorgängerzeiger setzen und g Wert merken oder anpassen
            previous_flat_map[successor] = current_node
            g_values_flat[successor] = tentative_g
            
            # f-Wert des Knotens in der Open List aktualisieren
            # bzw. Knoten mit f-Wert in die Open List einfügen
            f = tentative_g + heuristic_flat_map[successor]
            if not len(openlist_succ_index): 
                openlist.append((f, successor))
                openlist.sort(reverse=True)
            else:
                openlist[openlist_succ_index[0]] = (f, successor)

            
    # diese Schleife wird durchlaufen bis entweder
    # - die optimale Lösung gefunden wurde oder
    # - feststeht, dass keine Lösung existiert
    while len(openlist) != 0:        
        # Knoten mit dem geringsten f-Wert aus der Open List entfernen
        current_node_f = openlist.pop()

        if current_node_f[1] % 1000 == 0:      
            clear_output(wait=True)
            display("Current node is", current_node_f[1])
        
        # Wurde das Ziel gefunden?
        if current_node_f[1] == goal:
            return previous_flat_map
        
        # Der aktuelle Knoten soll durch nachfolgende Funktionen
        # nicht weiter untersucht werden, damit keine Zyklen entstehen
        closedlist = np.append(closedlist, current_node_f[1])
        
        # Wenn das Ziel noch nicht gefunden wurde: Nachfolgeknoten
        # des aktuellen Knotens auf die Open List setzen
        expand_node(current_node_f[1])
    
    # die Open List ist leer, es existiert kein Pfad zum Ziel
    return "NoPathFound"

In [12]:
def mark_previous_recursive(p_map, curr):
    if curr == 0:
        return p_map;
    previous = p_map[curr]
    p_map[curr] = 0
    return mark_previous(p_map, previous)

def mark_previous(p_map, curr):
    while True:
        if curr == 0:
            return p_map;
        previous = p_map[curr]
        p_map[curr] = 0
        curr = previous

In [13]:
def sum_previous_recursive(p_map, curr, levels):
    if curr == 0:
        return 0
    previous = p_map[curr]
    return levels[curr] + sum_previous(p_map, previous, levels)


def sum_previous(p_map, curr, levels):
    all_sum = 0;
    while True:
        if curr == 0:
            return all_sum
        previous = p_map[curr]
        all_sum += levels[curr]
        curr = previous

In [41]:
def print_a_star_info(levels, p_map, print_map=False, goal=None):
    if not goal:
        goal = len(levels)*len(levels[0]) -1
    print("\nbest way: ", sum_previous(p_map.copy(), goal, levels.flatten())) 
    print("visited: ", len(p_map[p_map != -1]))
    print("not visited: ", len(p_map[p_map == -1]))
    if print_map:
        print(mark_previous(p_map.copy(), goal).reshape((len(levels),len(levels[0]))))
    return mark_previous(p_map.copy(), goal).reshape((len(levels),len(levels[0])))

In [15]:
previous_map_test = a_star(test_levels)
_ = print_a_star_info(test_levels, previous_map_test, True)

'Current node is'

0


best way:  40
visited:  100
not visited:  0
[[ 0  0  1  2  3  4  5  6  7  8]
 [ 0  1 11  3 13 14 26 27 28 18]
 [ 0  0  0  0  0  0  0 26 27 28]
 [20 21 22 23 33 25  0  0 28 49]
 [30 31 41 33 43 35 36  0  0 48]
 [40 41 51 52 44 45 55 47  0 49]
 [50 60 61 62 54 55 65 66  0 68]
 [60 70 71 72 73 65 75 76  0 69]
 [70 71 72 73 83 75 76 77  0  0]
 [80 81 91 92 93 85 95 96 88  0]]


In [16]:
previous_map = a_star(initial_levels)

'Current node is'

9000

In [17]:
bestway = print_a_star_info(initial_levels, previous_map)
np.savetxt("../tmp/a_star_way.txt", (bestway == 0)+0, fmt='%u')


best way:  619
visited:  10000
not visited:  0


# Part 2

In [18]:
def enlarge_map(i_map):
    new_map = i_map.copy()
    for i in range(4):
        new_vals = i_map + 1 + i;
        new_map = np.append(new_map, new_vals, axis=0)        
    copy_map = new_map.copy()
    for i in range(4):
        new_vals = copy_map + 1 + i;
        new_map = np.append(new_map, new_vals, axis=1)
    return (new_map -1) % 9 + 1;

In [19]:
test_levels_pt2 = enlarge_map(test_levels)
previous_map_test_pt2 = a_star(test_levels_pt2)
bestway_test_pt2 = print_a_star_info(test_levels_pt2, previous_map_test_pt2, False)
np.savetxt("../tmp/a_star_way_test_pt2.txt", (bestway == 0)+0, fmt='%u')

'Current node is'

2000


best way:  315
visited:  2500
not visited:  0


In [20]:
initial_levels_pt2 = enlarge_map(initial_levels)

In [21]:
previous_map_initial_pt2 = a_star(initial_levels_pt2)

'Current node is'

249000

In [22]:
bestway_initial_pt2 = print_a_star_info(initial_levels_pt2, previous_map_initial_pt2, False)
np.savetxt("../tmp/a_star_way_inital_pt2.txt", (bestway_initial_pt2 == 0)+0, fmt='%u')


best way:  2922
visited:  249993
not visited:  7


In [23]:
np.savetxt("../tmp/initial_levels_pt2.txt", initial_levels_pt2, fmt='%u')

In [24]:
np.savetxt("../tmp/previous_map_initial_pt2.txt", previous_map_initial_pt2.reshape((len(initial_levels_pt2),len(initial_levels_pt2[0]))), fmt='%u')

In [25]:
np.savetxt("../tmp/heuristics_pt2.txt", compute_heuristic_map(initial_levels_pt2), fmt='%u')

# Experiments

In [42]:
goal = len(initial_levels_pt2[0]) * (len(initial_levels_pt2) - 1)

In [43]:
previous_map_initial_pt2_down = a_star(initial_levels_pt2, goal)

'Current node is'

249000

In [44]:
bestway_initial_pt2_down = print_a_star_info(initial_levels_pt2, previous_map_initial_pt2_down, False, goal)
np.savetxt("../tmp/a_star_way_inital_pt2_down.txt", (bestway_initial_pt2_down == 0)+0, fmt='%u')


best way:  1742
visited:  208233
not visited:  41767
